In [1]:
from tensorflow.keras.layers import Conv2D, Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np

In [2]:
#Prepare data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = np.reshape(x_train/255., (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test/255., (len(x_test), 28, 28, 1))

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [3]:
#Model creation function
def build_model():
    model = Sequential()
    model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'Same', 
                    activation ='relu', input_shape = (28,28,1)))
    model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                    activation ='relu'))
    model.add(Flatten())
    model.add(Dense(128, activation = "relu"))
    model.add(Dense(10, activation = "softmax"))

    model.compile(
        optimizer='adam',
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [14]:
clf = KerasClassifier(build_fn = build_model)

In [6]:
%%time

clf.fit(x_train,y_train,epochs=10);

Epoch 1/10
1875/1875 [==============================] - 122s 65ms/step - loss: 0.1150 - accuracy: 0.9656
Epoch 2/10
1875/1875 [==============================] - 122s 65ms/step - loss: 0.0383 - accuracy: 0.9883
Epoch 3/10
1875/1875 [==============================] - 120s 64ms/step - loss: 0.0229 - accuracy: 0.9924
Epoch 4/10
1875/1875 [==============================] - 120s 64ms/step - loss: 0.0145 - accuracy: 0.9955
Epoch 5/10
1875/1875 [==============================] - 120s 64ms/step - loss: 0.0106 - accuracy: 0.9963
Epoch 6/10
1875/1875 [==============================] - 124s 66ms/step - loss: 0.0082 - accuracy: 0.9974
Epoch 7/10
1875/1875 [==============================] - 123s 65ms/step - loss: 0.0078 - accuracy: 0.9974
Epoch 8/10
1875/1875 [==============================] - 123s 66ms/step - loss: 0.0061 - accuracy: 0.9978
Epoch 9/10
1875/1875 [==============================] - 134s 71ms/step - loss: 0.0050 - accuracy: 0.9984
Epoch 10/10
1875/1875 [==============================] 

In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [7]:
from dask.distributed import Client
import joblib

tritonCluster = Client("tcp://192.168.86.34:8786")

In [10]:
tritonCluster

Client Scheduler: tcp://192.168.86.34:8786 Dashboard: http://192.168.86.34:8787/status,Cluster Workers: 2 Cores: 14 Memory: 21.82 GB


In [11]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(verbose=1)

In [12]:
import joblib

In [17]:
%%time

with joblib.parallel_backend('dask'):
    clf.fit(x_train,y_train,epochs=10)
    predictions = clf.predict(x_test)

Epoch 1/10
1875/1875 [==============================] - 79s 42ms/step - loss: 0.1108 - accuracy: 0.9662
Epoch 2/10
1875/1875 [==============================] - 79s 42ms/step - loss: 0.0364 - accuracy: 0.9887
Epoch 3/10
1875/1875 [==============================] - 79s 42ms/step - loss: 0.0215 - accuracy: 0.9934
Epoch 4/10
1875/1875 [==============================] - 98s 52ms/step - loss: 0.0157 - accuracy: 0.9951
Epoch 5/10
1875/1875 [==============================] - 103s 55ms/step - loss: 0.0109 - accuracy: 0.9966
Epoch 6/10
1875/1875 [==============================] - 93s 50ms/step - loss: 0.0087 - accuracy: 0.9972
Epoch 7/10
1875/1875 [==============================] - 81s 43ms/step - loss: 0.0077 - accuracy: 0.9975
Epoch 8/10
1875/1875 [==============================] - 83s 44ms/step - loss: 0.0055 - accuracy: 0.9982
Epoch 9/10
1875/1875 [==============================] - 80s 43ms/step - loss: 0.0055 - accuracy: 0.9983
Epoch 10/10
1875/1875 [==============================] - 83s 44